In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft

In [2]:
theta = 0.132328; kappa = 10.980797; sigma = 1.7; rho = -0.351560; v0 = 0.065690
spot, K = 659.37, 758.28
alp = 0.75
s = np.log(spot)
feller_cond = 2*kappa*theta > sigma**2
print("Feller condition is ", feller_cond)

Feller condition is  True


In [3]:
def HestonMC (spot, v0, rho, kappa, theta, sigma, T, num_steps, r=0.0, return_log = True):
    # Generate a path
    vt    = np.zeros(num_steps)
    vt[0] = v0
    logSt = np.zeros(num_steps)
    logSt[0] = np.log(spot)
    dt = T / num_steps

    # Milstein scheme for volatility
    for t in range(1,num_steps):
        # Generate random Brownian Motion
        dW_indep = np.random.normal(0.0,1.0,2)
        dW_v = dW_indep[0]
        dW_logS = rho*dW_indep[0] + np.sqrt(1-rho**2)*dW_indep[1] 
        vt[t] = vt[t-1] + kappa*(theta-vt[t-1])*dt + sigma* np.sqrt(vt[t-1]*dt)*dW_v + sigma**2/4.*dt*(dW_v**2-1.)
        if vt[t] < 0.0:
            vt[t] = 0.0
        logSt[t] = logSt[t-1] + (r - vt[t-1]/2.)*dt + np.sqrt(vt[t-1]*dt)*dW_logS

    if return_log:
        return logSt, vt
    else:        
        St= np.exp(logSt)
        return St, vt

In [4]:
u = 1.0 
k = np.log(K)
x = np.log(spot)
r = 0.0
num_paths = 5000
payoff_sum = 0.0
T = 1.0
num_steps = 252

In [5]:
def get_chf(x_T,u):
    return np.exp(u*1j*x_T)

"""
Compute E[e^{1j*u*x}]
x: log(S)
u: variable for the chf
v0: v0
r: risk-free rate
rho: correlation
ka: kappa
th: theta
sig: sigma
"""
def get_Heston_chf(x, u, v0, r, rho, ka, th, sig, T):
    gam = np.sqrt(sig**2*u*(u+1j) + (ka-1j*rho*sig*u)**2)
    half_gam_T = gam*T/2.0
#    print("half gam ", half_gam_T)
    coth = np.cosh(half_gam_T)/np.sinh(half_gam_T)
    log_numerator = ka*th*T*(ka-1j*rho*sig*u)/sig**2 + 1j*u*(T*r+x) - (u*(u+1j)*v0)/(gam*coth + ka - 1j*rho*sig*u)
    numerator = np.exp(log_numerator)
    denominator = (np.cosh(half_gam_T) + (ka-1j*rho*sig*u)/gam*np.sinh(half_gam_T))**(2.0*ka*th/sig**2)
    return numerator/denominator

def get_damped_Heston_chf(x, v, v0, r, rho, ka, th, sig, T, alp):
    chf_inp = v - (alp+1.0)*1j
    numerator = np.exp(-r*T) * get_Heston_chf(x, chf_inp, v0, r, rho, ka, th, sig, T)
    denominator = alp**2 + alp - v**2 +1j*(2.0*alp +1.0)*v
    return numerator / denominator

In [6]:
heston_chf_mc = 0.0
damped_heston_chf_mc = 0.0
mc_call_price = 0.0
for i in range(num_paths):
    logS_last = HestonMC(spot, v0, rho, kappa, theta, sigma, T, num_steps, r=r, return_log = True)[0][-1]
    heston_chf_mc += get_chf(logS_last,u)
    damped_heston_chf_mc += heston_chf_mc 
    mc_call_price += max(0., np.exp(logS_last)-K)
damped_heston_chf_mc *= np.exp(alp*k)
heston_chf_mc /= num_paths
damped_heston_chf_mc /= num_paths
mc_call_price /= num_paths
print('chf from mc ', heston_chf_mc)
print('analytic chf ', get_Heston_chf(x, u, v0, r, rho, kappa, theta, sigma, T))
print('call from mc ', mc_call_price)
print(spot, K, v0, rho)

chf from mc  (0.9269540211988475+0.1411859786965216j)
analytic chf  (0.9257979660695421+0.13883412520889077j)
call from mc  53.71773020615279
659.37 758.28 0.06569 -0.35156


In [7]:
print("damped heston ", get_damped_Heston_chf(x, 0.1, v0, r, rho, kappa, theta, sigma, T, alp))
print("mc ", damped_heston_chf_mc)

damped heston  (62119.916642292934+31855.743812080007j)
mc  (335215.7276293936+50667.94725164735j)


In [8]:
N = 2048*4
u_list = list(range(N-1))
dk = k * 0.005 # zeta
dv = 2.0*np.pi/(N*dk) # nu
k_list = [s + dk*u_elem - N*dk/2.0 for u_elem in u_list]
v_list = range(N-1) * dv
x_list = [] # values in frequency domain
for (j,vj) in enumerate(v_list): # j=0 to N-1
    if j==0:
        kroneker_delta = 1.0
    else:
        kroneker_delta = 0.0        
    coeff = dv/3.0*(3.0 + (-1)**j - kroneker_delta)
    xj = np.exp(1j*(N/2.0*dk-s)*vj)*coeff*get_damped_Heston_chf(x, vj, v0, r, rho, kappa, theta, sigma, T, alp)
    x_list.append(xj)
    
fft_prices = fft(x_list).real
fft_prices *= np.exp(-alp*k)/np.pi

In [9]:
k_list[int(N/2)]

6.491284833697279

In [10]:
fft_prices[int(N/2)]

78.74617904128196

In [11]:
k

6.631052710573364

In [12]:
k_list[int(N/2)+1]

6.524440097250135